In [1]:
import pandas as pd
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn import metrics
from collections import defaultdict
import joblib


import sys

import time
start_time = time.time()


infile='/Users/irffanalahi/Research/Research_update/SoftRD/largerEXP/BL22genepromdelta.7/Realdata/RF/BL22_genepromSM_0.7.txt_TRAININGbestref.txt_result_dupindex_binnedstats.pkl_totalformatteddata.pkl'


#gtcol='label2'

##### carefully input the model folder#####

modelfol='/Users/irffanalahi/Research/Research_update/SoftRD/largerEXP/BL22genepromdelta.7/insilico/training/RF/BL22_genepromSM_0.7.txt_TRAININGbestref.txt_result_dupindex_binnedstats.pkl_totalformatteddata.pkl_model'


indf=pd.read_pickle(infile)

indf.head()

,index,NaiveCD4-others,NaiveCD8-others,nB-others,NK-others,PC-others,Mono-others,M0-others,M1-others,M2-others,...,total_cpg,LENhypoCpG_BY_total_cpg,maxscoredCT_beforeCpGweight,maxscore_beforeCpGweight,maxscoredCT,maxscore,deltabasedfragassignment,filename,deltaMedian,deltaMean
0,A00354:599:H3KWLDMXY:2:1101:18738:5102,0.415913,-0.121865,-0.208817,-0.167960,-0.172151,-0.012913,0.773151,0.745563,-0.025484,...,1.0,1.00,M0-others,0.773151,M0-others,0.773151,M0-others,healthy_NU-49-bulk-PBMC_sorted_onlyregionIN_BL...,-0.102746,2.270911e-17
1,A00354:599:H3KWLDMXY:2:1101:22779:13385,-0.140095,-0.096794,-0.171175,-0.173270,0.129143,-0.086317,0.332730,0.601968,0.825111,...,1.0,1.00,M2-others,0.825111,M2-others,0.825111,M2-others,healthy_NU-49-bulk-PBMC_sorted_onlyregionIN_BL...,-0.100984,-3.280204e-17
2,A00354:599:H3KWLDMXY:2:1101:24044:2707,-0.156056,-0.156056,-0.114151,-0.110310,-0.104024,-0.026849,0.768992,0.408611,0.431310,...,1.0,1.00,M0-others,0.768992,M0-others,0.768992,M0-others,healthy_NU-49-bulk-PBMC_sorted_onlyregionIN_BL...,-0.091103,1.387779e-17
3,A00354:599:H3KWLDMXY:2:1101:25138:24392,-0.061063,-0.046746,-0.061762,-0.043603,0.968746,-0.055825,-0.027889,-0.071540,-0.003095,...,4.0,0.25,PC-others,0.968746,PC-others,0.242187,PC-others,healthy_NU-49-bulk-PBMC_sorted_onlyregionIN_BL...,-0.046135,7.569702e-17
4,A00354:599:H3KWLDMXY:2:1101:30987:11537,-0.158698,0.420984,-0.154857,-0.117492,-0.072444,-0.152762,-0.142286,-0.172667,-0.137746,...,2.0,0.50,cm8-others,0.826937,cm8-others,0.413468,cm8-others,healthy_NU-49-bulk-PBMC_sorted_onlyregionIN_BL...,-0.140016,-8.452834e-17


In [2]:
#BL22

features=['NaiveCD4-others','NaiveCD8-others','nB-others','NK-others','PC-others','Mono-others','M0-others','M1-others','M2-others','iDC-others','mDC-others','PMN-others','cm8-others','em8-others','Eo-others','Tregs-others','em4-others','ed8-others','Mg-others','cm4-others','Er-others','mB-others','LENhypoCpG', 'LENhyperCpG', 'total_cpg', 'LENhypoCpG_BY_total_cpg','deltaMedian', 'deltaMean']




#BL14
#features=['CD4-others', 'CD8-others', 'nB-others', 'NK-others','Mn-others', 'mNeu-others', 'm8-others', 'DC-others', 'Eo-others','Tr-others', 'm4-others', 'Mg-others', 'Er-others', 'mB-others','LENhypoCpG', 'LENhyperCpG', 'total_cpg', 'LENhypoCpG_BY_total_cpg','deltaMedian', 'deltaMean']

In [3]:
#BL22
scorecolumns=['NaiveCD4-others','NaiveCD8-others','nB-others','NK-others','PC-others','Mono-others','M0-others','M1-others','M2-others','iDC-others','mDC-others','PMN-others','cm8-others','em8-others','Eo-others','Tregs-others','em4-others','ed8-others','Mg-others','cm4-others','Er-others','mB-others']



#BL14
#scorecolumns=['CD4-others', 'CD8-others', 'nB-others', 'NK-others','Mn-others', 'mNeu-others', 'm8-others', 'DC-others', 'Eo-others','Tr-others', 'm4-others', 'Mg-others', 'Er-others', 'mB-others']

In [4]:
def calscore(fgrouped,fname):
    celltype_probadded=defaultdict(list)
    
    celltype_proportiondict=defaultdict(list)
    

    for score in scorecolumns:

        tempct=score.replace('-others','')



        indfsingleCT=fgrouped[fgrouped['deltabasedfragassignment']==score].copy()





    
        testX=indfsingleCT[features]

        RDF_classifier =joblib.load(modelfol+"/"+tempct+".pkl")
        
        
        predictedCT=(RDF_classifier.predict(testX)).tolist()
        predictedTARGETCT=predictedCT.count(1)
        predictedNON_TARGETCT=predictedCT.count(0)
        
        totalpredictedCT=len(predictedCT)
        
        if (predictedTARGETCT+predictedNON_TARGETCT) != totalpredictedCT:
            print ("counting error. Exiting")
            sys.exit(1)
        
        
        cellularproportion=1.0*predictedTARGETCT/totalpredictedCT
        
        
        celltype_proportiondict[tempct]=celltype_proportiondict[tempct]+[cellularproportion]
        

        

        predict_test=RDF_classifier.predict_proba(testX)
        
        tempctprob=(predict_test[:,1]).tolist()
        
        tempct_sumprob=sum(tempctprob)
        
        celltype_probadded[tempct]=celltype_probadded[tempct]+[tempct_sumprob]
        
        
 
        
    
    
    celltype_proportiondictDF=pd.DataFrame.from_dict(celltype_proportiondict)
    celltype_proportiondictDF['Mixture']=fname
    
    celltype_probaddeddf=pd.DataFrame.from_dict(celltype_probadded)
    celltype_probaddeddf['Mixture']=fname
    return celltype_proportiondictDF, celltype_probaddeddf
    

In [5]:
filegrouped=indf.groupby('filename')

probaddedallfilelist=[]
ctproportionallfilelist=[]
for name, group in filegrouped:
    tempctproportion,tempprobadded=calscore(group,name)
    
    ctproportionallfilelist.append(tempctproportion)
    probaddedallfilelist.append(tempprobadded)
    

In [6]:
ctproportionallfilelistDF=pd.concat(ctproportionallfilelist)
ctproportionallfilelistDF.set_index('Mixture',inplace=True)
probaddedallfilelistDF=pd.concat(probaddedallfilelist)
probaddedallfilelistDF.set_index('Mixture',inplace=True)

In [7]:
ctproportionallfilelistDFNORMAL=ctproportionallfilelistDF.div(ctproportionallfilelistDF.sum(axis=1), axis=0)

In [8]:
ctproportionallfilelistDF.to_csv(infile+"_RFctprop_CSxOut.txt",sep="\t")
ctproportionallfilelistDFNORMAL.to_csv(infile+"_RFctpropNORMALIZED_CSxOut.txt",sep="\t")

probaddedallfilelistDF.to_csv(infile+"_RFprob_CSxOut.txt",sep="\t")

In [9]:
end_time = time.time()

time_elapsed = (end_time - start_time)

print(time_elapsed)

723.3606390953064
